# 9A_make_mpra_tracks

copy of 8B_make_rna_tracks

2/16/2021

how to make rna tracks : will be displaying TPM values on exons in hg19

Option 1: Wiggle (not done b/c wiggle not easy to process)
1. make wiggle file (https://genome.ucsc.edu/goldenPath/help/wiggle.html)
2. convert wig to BigWig (https://genome.ucsc.edu/goldenpath/help/bigWig.html)

Option 2: Bedgraph
1. make bedgraph (https://genome.ucsc.edu/goldenpath/help/bedgraph.html)
2. convert bedgraph to BigWig (https://genome.ucsc.edu/goldenpath/help/bigWig.html)
    To create a bigWig track from a bedGraph file, follow these steps:

1. Create a bedGraph format file following the directions here. When converting a bedGraph file to a bigWig file, you are limited to one track of data in your input file; therefore, you must create a separate bedGraph file for each data track.
2. Remove any existing track or browser lines from your bedGraph file so that it contains only data.
3. Download the bedGraphToBigWig program from the binary utilities directory.
4. Use the fetchChromSizes script from the same directory to create the chrom.sizes file for the UCSC database with which you are working (e.g., hg19). If the assembly genNom is hosted by UCSC, chrom.sizes can be a URL like http://hgdownload.soe.ucsc.edu/goldenPath/genNom/bigZips/genNom.chrom.sizes
5. Use the bedGraphToBigWig utility to create a bigWig file from your bedGraph file:
    `bedGraphToBigWig in.bedGraph chrom.sizes myBigWig.bw`
(Note that the bedGraphToBigWig program DOES NOT accept gzipped bedGraph input files.)
6. Move the newly created bigWig file (myBigWig.bw) to a web-accessible http, https, or ftp location.
7. Paste the URL into the custom track entry form or construct a custom track using a single track line.
8. Paste the custom track line into the text box on the custom track management page.



In [14]:
import pandas as pd
import os, glob, re
import subprocess
import pybedtools
import numpy as np

# 0. get files
- exon file
- rna seq tpm matrix file
- chrom size file

In [21]:
save_dir = '../data/processed/mpra_bigwigs'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
if not os.path.exists(os.path.join(save_dir, 'bedgraphs')):
    os.makedirs(os.path.join(save_dir, 'bedgraphs'))
if not os.path.exists(os.path.join(save_dir, 'bigwigs')):
    os.makedirs(os.path.join(save_dir, 'bigwigs'))

In [2]:
size_file = '../data/external/chrom_hg19.sizes'
exon_bed = '../data/external/gencode.v19.exon.bed'

In [3]:
exon_df = pybedtools.BedTool(exon_bed).to_dataframe()
exon_df[:5]

,chrom,start,end,name
0,chr1,11869,12227,DDX11L1
1,chr1,12613,12721,DDX11L1
2,chr1,13221,14409,DDX11L1
3,chr1,11872,12227,DDX11L1
4,chr1,12613,12721,DDX11L1


In [4]:
# rna_file = '../data/interim/rna/tissue_tpm_sym.csv'
# rna_df = pd.read_csv(rna_file)
# print(rna_df.shape)
# rna_df[:5]

In [25]:
mpra_df = pd.read_excel('/Users/mguo123/Google Drive/1_khavari/noncancer_project/miseq/novogene_071420/figure_tables/lib_df_annon_mprasig_FINAL_v020521.xlsx')
mpra_df_sel = mpra_df[['rowname','Chr','Position','Ref/Alt','Linked_SNP','diseases',
        'mpra_tissue',	'mpra_pval_str',	'mpra_pval_mean',
         'mpra_logfc_str',	'mpra_logfc_mean',	'bool_mpra_sig']]
mpra_df_sel['end'] = mpra_df_sel.Position +1
mpra_df_sel['neglog10pval'] = mpra_df_sel.mpra_pval_mean.apply(lambda x:-np.log10(x))
mpra_df_sel = mpra_df_sel[['Chr','Position','end','neglog10pval', 'mpra_logfc_mean','mpra_tissue','Linked_SNP','diseases']]
mpra_df_sel.columns = ['chr','start','stop','name', 'mpra_logfc_mean','mpra_tissue','Linked_SNP','diseases']
mpra_df_sel.to_csv(os.path.join(save_dir, 'mpra_df_sel.csv'))
mpra_df_sel[:5]

/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,chr,start,stop,name,mpra_logfc_mean,mpra_tissue,Linked_SNP,diseases
0,chr5,96161942,96161943,2.500462,-0.922195,H9D10|H9D10|SLC|SLC,rs152280,MDD
1,chr22,42347060,42347061,3.448500,-0.766147,H9D10|H9D10,rs5996094,SCZ
2,chr7,28799411,28799412,3.678498,-1.074166,H9D0|H9D0|H9D2|H9D2|H9D4|H9D4|HEK293|HEK293|SL...,rs10227088,GAD
3,chr7,1962163,1962164,2.978666,-1.456217,HEK293|HEK293,rs10268609,BPD
4,chr7,1915282,1915283,3.434151,-1.292504,H9D0|H9D0|H9D10|H9D10|H9D4|H9D4|HEK293|HEK293|...,rs7788921,BPD


In [17]:
tissues = ['AST', 'SL',  'SLC','H9D0', 'H9D2', 'H9D4','H9D10','HEK293']

In [41]:
# # bedgraph defintion lines
# definition_str = "track type=bedGraph name={track_label:s} description=center_label visibility=display_mode color=r,g,b altColor=r,g,b priority=priority autoScale=on|off alwaysZero=on|off gridDefault=on|off maxHeightPixels=max:default:min graphType=bar|points viewLimits=lower:upper yLineMark=real-value yLineOnOff=on|off windowingFunction=maximum|mean|minimum smoothingWindow=off|2-16"
# definition_str.format(track_label='test', )

In [42]:
# txt = "For only {price:.2f} dollars!"
# print(txt.format(price = 49))

# 1A. make bedGraph (no brower or track lines) per tissue

space delimited
`chrom chromStart chromEnd dataValue`

example:

```
chr19 49302000 49302300 -1.0
chr19 49302300 49302600 -0.75
```

In [30]:
for tissue in tissues:
    print('*****', tissue)
    mpra_df_tissue = mpra_df_sel[mpra_df_sel.mpra_tissue.str.contains(tissue)]
    
    mpra_df_tissue = mpra_df_tissue[['chr','start','stop','name']]
    mpra_df_bed = pybedtools.BedTool.from_dataframe(mpra_df_tissue).sort()#.merge(c=4,o='sum')
    mpra_df_tissue = mpra_df_bed.to_dataframe()
    print('size mpra_df_bed sorted ',mpra_df_tissue.shape)
    mpra_df_tissue.to_csv(os.path.join(save_dir,'bedgraphs', tissue+'_unsorted.bedGraph'),index=False, header=False,sep=' ')    
    

***** AST
size mpra_df_bed sorted  (66, 4)
***** SL
size mpra_df_bed sorted  (355, 4)
***** SLC
size mpra_df_bed sorted  (335, 4)
***** H9D0
size mpra_df_bed sorted  (105, 4)
***** H9D2
size mpra_df_bed sorted  (86, 4)
***** H9D4
size mpra_df_bed sorted  (188, 4)
***** H9D10
size mpra_df_bed sorted  (153, 4)
***** HEK293
size mpra_df_bed sorted  (273, 4)


In [31]:
mpra_df_tissue[:5]

,chrom,start,end,name
0,chr1,7292700,7292701,3.261096
1,chr1,8033468,8033469,3.356089
2,chr1,8123684,8123685,2.787682
3,chr1,8484529,8484530,2.264198
4,chr1,8484823,8484824,7.523371


# 1B. sort bedGraphs

`sort -k1,1 -k2,2n unsorted.bedGraph > sorted.bedGraph`

In [32]:
for tissue in tissues:
    unsort_file = os.path.join(save_dir, 'bedgraphs',tissue+'_unsorted.bedGraph')
    sort_file = os.path.join(save_dir,'bedgraphs', tissue+'_sorted.bedGraph')
    sort_cmd = 'sort -k1,1 -k2,2n "{}" > "{}"'.format(unsort_file,sort_file)
    print(sort_cmd)
    subprocess.call(sort_cmd,shell=True)

sort -k1,1 -k2,2n "../data/processed/mpra_bigwigs/bedgraphs/AST_unsorted.bedGraph" > "../data/processed/mpra_bigwigs/bedgraphs/AST_sorted.bedGraph"
sort -k1,1 -k2,2n "../data/processed/mpra_bigwigs/bedgraphs/SL_unsorted.bedGraph" > "../data/processed/mpra_bigwigs/bedgraphs/SL_sorted.bedGraph"
sort -k1,1 -k2,2n "../data/processed/mpra_bigwigs/bedgraphs/SLC_unsorted.bedGraph" > "../data/processed/mpra_bigwigs/bedgraphs/SLC_sorted.bedGraph"
sort -k1,1 -k2,2n "../data/processed/mpra_bigwigs/bedgraphs/H9D0_unsorted.bedGraph" > "../data/processed/mpra_bigwigs/bedgraphs/H9D0_sorted.bedGraph"
sort -k1,1 -k2,2n "../data/processed/mpra_bigwigs/bedgraphs/H9D2_unsorted.bedGraph" > "../data/processed/mpra_bigwigs/bedgraphs/H9D2_sorted.bedGraph"
sort -k1,1 -k2,2n "../data/processed/mpra_bigwigs/bedgraphs/H9D4_unsorted.bedGraph" > "../data/processed/mpra_bigwigs/bedgraphs/H9D4_sorted.bedGraph"
sort -k1,1 -k2,2n "../data/processed/mpra_bigwigs/bedgraphs/H9D10_unsorted.bedGraph" > "../data/processed/mp

# 2. run `bedGraphToBigWig`
cmd: `bedGraphToBigWig in.bedGraph chrom.sizes myBigWig.bw`

In [33]:
bedGraphToBigWig_cmd = '/Users/mguo123/Documents/genome_browser_tools/bedGraphToBigWig'


In [34]:
for tissue in tissues:
    bedgraph_file = os.path.join(save_dir,'bedgraphs', tissue+'_sorted.bedGraph')
    bigwig_file = os.path.join(save_dir,'bigwigs', tissue+'.bw')
    cmd = '{} {} {} {}'.format(bedGraphToBigWig_cmd,bedgraph_file, size_file,bigwig_file )
    print(cmd)
    subprocess.call(cmd,shell=True)

/Users/mguo123/Documents/genome_browser_tools/bedGraphToBigWig ../data/processed/mpra_bigwigs/bedgraphs/AST_sorted.bedGraph ../data/external/chrom_hg19.sizes ../data/processed/mpra_bigwigs/bigwigs/AST.bw
/Users/mguo123/Documents/genome_browser_tools/bedGraphToBigWig ../data/processed/mpra_bigwigs/bedgraphs/SL_sorted.bedGraph ../data/external/chrom_hg19.sizes ../data/processed/mpra_bigwigs/bigwigs/SL.bw
/Users/mguo123/Documents/genome_browser_tools/bedGraphToBigWig ../data/processed/mpra_bigwigs/bedgraphs/SLC_sorted.bedGraph ../data/external/chrom_hg19.sizes ../data/processed/mpra_bigwigs/bigwigs/SLC.bw
/Users/mguo123/Documents/genome_browser_tools/bedGraphToBigWig ../data/processed/mpra_bigwigs/bedgraphs/H9D0_sorted.bedGraph ../data/external/chrom_hg19.sizes ../data/processed/mpra_bigwigs/bigwigs/H9D0.bw
/Users/mguo123/Documents/genome_browser_tools/bedGraphToBigWig ../data/processed/mpra_bigwigs/bedgraphs/H9D2_sorted.bedGraph ../data/external/chrom_hg19.sizes ../data/processed/mpra_bi

# 3. make hub.config.json

manually made

final folder for upload: `/Users/mguo123/Documents/pan_omics_psych/data/processed/mpra_bigwigs/bigwigs`

# 4. checking
